In [ ]:
import rasterio
import numpy as np
import cv2
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
import torch
import matplotlib.pyplot as plt
import gc

gc.collect()
torch.cuda.empty_cache()

# Load the SAM model
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
# sam_checkpoint = "sam_vit_l_0b3195.pth"
# model_type = "vit_l"
# sam_checkpoint = "sam_vit_b_01ec64.pth"
# model_type = "vit_b"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

# Function to get segmentation mask (adjust as needed for SAM usage)
def get_segmentation(patch):
    mask_generator = SamAutomaticMaskGenerator(sam)
    masks = mask_generator.generate(patch)
    return masks

def pixel_to_tm2(pixel_coords, transform):
    tm2_coords = [transform * (x, y) for x, y in pixel_coords]
    return np.array(tm2_coords)

def is_contour_on_edge(contour, patch_size):
    """
    Check if a contour touches the edge of the patch.
    :param contour: np.ndarray of contour points
    :param patch_size: Size of the patch (assuming square patch)
    :return: Boolean indicating if the contour touches the edge
    """
    x_min, y_min = contour.min(axis=0)
    x_max, y_max = contour.max(axis=0)
    if x_min <= 0 or y_min <= 0 or x_max >= patch_size - 1 or y_max >= patch_size - 1:
        return True
    return False

# Function to process each patch
def process_patch(patch, transform):
    # Get the segmentation mask for the patch
    masks = get_segmentation(patch)

    polygons = []
    for mask in masks:
        mask_np = mask['segmentation'].astype(np.uint8)
        contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            contour = contour.squeeze().reshape(-1, 2)
            if len(contour) >= 3 and not is_contour_on_edge(contour, patch_size):  # at least 3 points to form a polygon
                # Convert contour coordinates to TM2
                tm2_coords = pixel_to_tm2(contour, transform)
                # Create Polygon object with TM2 coordinates
                polygons.append(Polygon(tm2_coords))
    return polygons

# Path to the large TIFF image
# image_path = "h:\\Yehmh\\FuShan\\202403\\202403_rgb.tif"
# image_path = 'h:\\Yehmh\DNDF\\86_1_clipped_image_2.tiff'
image_path = 'h:\\Yehmh\DNDF\\Clipped_image\\101_1_clipped_image.tif'

# Initialize an empty GeoDataFrame
gdfs = []

# Read and process the image in patches
with rasterio.open(image_path) as src:
    patch_size = 256  # Define your patch size
    overlap = int(patch_size*0.5)  # Define your patch step
    # overlap = 0
    print(src.width, src.height)
    for i in range(0, src.width, patch_size - overlap):
        for j in range(0, src.height, patch_size - overlap):
            window = rasterio.windows.Window(i, j, patch_size, patch_size)
            transform = src.window_transform(window)
            patch = src.read(window=window)
            print(patch.shape)
            patch = np.moveaxis(patch, 0, -1)  # Move channels to last dimension if necessary
            print(patch.shape)
            patch = patch[:,:,:3]
            print(i, j, patch.shape)
            # plt.imshow(patch)
            # Process the patch
            polygons = process_patch(patch, transform)
            local_gdf = gpd.GeoDataFrame(geometry=polygons, crs=src.crs)
            gdfs.append(local_gdf)

# Concatenate all GeoDataFrames
gdf = pd.concat(gdfs, ignore_index=True)

# Merge polygons that overlap or are very close to each other
# gdf = gdf.dissolve()

# Save the resulting shapefile
# shapefile_path = "h:\\Yehmh\\FuShan\\202403\\202403_rgb_masks.shp"
# shapefile_path = 'h:\\Yehmh\DNDF\\Clipped_image\\masks\\86_1_masks_patch1024_noEdge_dissolve.shp'
shapefile_path = 'h:\\Yehmh\DNDF\\Clipped_image\\masks\\101_1_masks_patch1024_noEdge_overlap_50.shp'
gdf.to_file(shapefile_path)

In [86]:
import geopandas as gpd

# Read the shapefile
gdf = gpd.read_file('h:\\Yehmh\DNDF\\86_1_masks_patch1024_noEdge.shp')

# Calculate the area of each polygon
gdf['area'] = gdf.geometry.area

# Filter out polygons with area less than 2
filtered_gdf = gdf[gdf['area'] >= 2]

# Save the filtered shapefile if needed
filtered_gdf.to_file('h:\\Yehmh\DNDF\\86_1_masks_patch1024_noEdge_2up.shp')

In [53]:
torch.cuda.empty_cache()